## Imports

In [1]:
import math
import os
import pprint
import logging

import nltk
import numpy as np
import torch
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator
from filelock import FileLock
from transformers import AdamW, get_scheduler, set_seed

from transformers.file_utils import is_offline_mode
from transformers.utils.versions import require_version

from args import parse_args
from data_loader import raw_data_loader, data_processor
from model_loader import model_loader
from rouge_s import py_rouge_scores
from scoring import bleu_scores, meteor_scores
from utils import label_smoothed_nll_loss, postprocess_text

/home/ubuntu/anaconda3/envs/hlt/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prep Model

In [2]:
class Args:
    output_dir = "./output/run_mask_info_samsum_bart_base"
    train_file = "./data/samsum/train_small.csv"
    validation_file = "./data/samsum/val_small.csv"
    test_file = "./data/samsum/test_small.csv"
    text_column = "dialogue"
    summary_column = "summary"
#     model_name_or_path = "t5-base"
    model_name_or_path = "./output/run_mask_info_samsum_bart_base/best"
    model_type = "bart"
    source_prefix = ""
    max_source_length = 1024
    min_target_length = 1
    max_target_length = 128
    learning_rate = 5e-5
    weight_decay = 1e-3
    label_smoothing = 0.1
    length_penalty = 1.0
    num_train_epochs = 4
    per_device_train_batch_size = 1
    gradient_accumulation_steps = 16
    per_device_eval_batch_size = 1
    per_device_test_batch_size = 1
    num_warmup_steps = 0
    cache_dir = "./output/cache"
    overwrite_cache = True
    seed = 12345
    
    ignore_pad_token_for_loss = True
    preprocessing_num_workers = None
    overwrite_cache = None
    num_beams = None
    pad_to_max_length = True
    config_name = None
    tokenizer_name = "t5-base"
    use_slow_tokenizer = True
    max_train_steps = None
    lr_scheduler_type = "linear"
    shuffle = False
    
args=Args()

In [3]:
# =  =  =  =  =  =  =  =  =  = Logging Setup =  =  =  =  =  =  =  =  =  =  =  = 
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

In [4]:
# =  =  =  =  =  =  =  =  =  = Pre-check Package Info =  =  =  =  =  =  =  =  =  =  =  = 
require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/summarization/requirements.txt")

try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    if is_offline_mode():
        raise LookupError(
            "Offline mode: run this script without TRANSFORMERS_OFFLINE first to download nltk data files"
        )
    with FileLock(".lock") as lock:
        nltk.download("punkt", quiet=True)

In [5]:
# = = = = = = = = = = = = = Main Process = = = = = = = = = = = = = = = = = =
# Initialize the accelerator. The accelerator will handle device placement for us.
accelerator = Accelerator()
logger.info(accelerator.state)

# Setup logging, we only want one process per machine to log things on the screen.
# accelerator.is_local_main_process is only True for one process per machine.
logger.setLevel(logging.INFO if accelerator.is_local_main_process else logging.ERROR)
if accelerator.is_local_main_process:
    #datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_info()
else:
    #datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()

# If passed along, set the training seed now.
if args.seed is not None:
    set_seed(args.seed)
    torch.backends.cudnn.enabled = False 
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

if accelerator.is_main_process:
    if args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)
accelerator.wait_for_everyone()

# load raw dataset
raw_datasets = raw_data_loader(args)

# load model (config, tokenizer, s2s model)
config, tokenizer, model = model_loader(accelerator, logger, args)

# data processor (for DataLoader)
dataloader, processed_dataset = data_processor(logger, args, accelerator, raw_datasets, tokenizer, model)
train_dataloader, eval_dataloader, test_dataloader = dataloader
train_dataset, _, _ = processed_dataset

07/27/2022 16:48:04 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Use FP16 precision: False

loading configuration file ./output/run_mask_info_samsum_bart_base/best/config.json
Model config BartConfig {
  "_name_or_path": "./output/run_mask_info_samsum_bart_base/best",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "for

In [6]:
unwrapped_model = accelerator.unwrap_model(model)
config          = config.from_pretrained(args.output_dir+'/best')
tokenizer       = tokenizer.from_pretrained(args.output_dir+'/best', config=config)
unwrapped_model = unwrapped_model.from_pretrained(args.output_dir+'/best', config=config)
model           = accelerator.prepare(unwrapped_model)

loading configuration file ./output/run_mask_info_samsum_bart_base/best/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "l

## Define Functions

In [7]:
# =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  = EVAL =  =  =  =  =  =  =  =  =  =  =  =  =  =  = 
def generate_new(input_text):
    model.eval()

    with torch.no_grad():
        tokenized = tokenizer([input_text], max_length=args.max_source_length, padding='max_length', truncation=True)
        inputs, mask = tokenized["input_ids"], tokenized["attention_mask"]
        inputs, mask = torch.tensor(inputs), torch.tensor(mask)
        inputs = inputs.to(device="cuda:0")
        mask = mask.to(device="cuda:0")

        generated_tokens = accelerator.unwrap_model(model).generate(inputs, attention_mask = mask)
        generated_tokens = accelerator.pad_across_processes(generated_tokens, dim=1, pad_index=tokenizer.pad_token_id)
        generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
        dialogue_output = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        
    return dialogue_output

In [8]:
def string_overlap(summary_list, utterance_list):
    count = 0
    
    for word in utterance_list:
        if word in summary_list:
            count += 1
    
    return count

In [9]:
def length_bucket(utterance_list):
    length = len(utterance_list)
    
    if length <= 4:
        return "S"
    if length > 10:
        return "L"
    
    return "M"

In [10]:
def generate_input(summary, dialogue, i):
    dialogue_sep = dialogue.split('\n')
    length = len(dialogue_sep)

    input_str = "Summary - " + summary + "\n" + "Dialogue - \n"

    speaker = dialogue_sep[i].split(':')[0]
    target = dialogue_sep[i].split(':')[1]

    speaker = "Speaker - " + speaker + "\n"

    overlap = string_overlap(summary.split(), target.split())
    total = len(summary.split())
    add_info = "Overlap - " + str(overlap) + ", Total - " + str(total) + "\n"

    length_info = "Length - " + length_bucket(target.split())

    temp_dialogue = dialogue_sep.copy()
    temp_dialogue[i] = "<mask>"
    temp_dialogue = '\n'.join(temp_dialogue)

    return input_str + temp_dialogue + '\n\n' + speaker + add_info + length_info

In [11]:
def generate_new_dialogue(dialogue, i, new_utterance):
    dialogue_sep = dialogue.split('\n')

    speaker = dialogue_sep[i].split(':')[0]
    target = dialogue_sep[i].split(':')[1]

    temp_dialogue = dialogue_sep.copy()
    temp_dialogue[i] = speaker + ":" + new_utterance
    temp_dialogue = '\n'.join(temp_dialogue)

    return temp_dialogue

## Generate New Dialogues

In [12]:
import random
import json

In [13]:
file_path = "data/dialogsum/dialogsum.train.jsonl"

In [14]:
data = []

with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

id_list       = [sample['fname'] for sample in data]
dialogue_list = [sample['dialogue'] for sample in data]
summary_list  = [sample['summary'] for sample in data]

In [15]:
dialogue = dialogue_list[123]
print(dialogue)

#Person1#: Do you have any experience working with a computer?
#Person2#: Yes. I have been a data entry operator for three years.
#Person1#: What kind of software can you use?
#Person2#: I have working knowledge of Windows and Dos. Actually, I'm quite familiar with both Java and C Programming Languages.
#Person1#: Do you have any other computer qualifications?
#Person2#: I have an ACRE certificate, GRADE 2.
#Person1#: Do you know how to use a PC to process the management information?
#Person2#: I'm sorry to say I'm not familiar with processing management information, but I'm sure I could learn quite quickly. It can't be too difficult, and I've got a quick mind. I can handle any problem you give me.


In [16]:
summary = summary_list[123]
print(summary)

#Person1# interviews #Person2# who has been a data entry operator for three years. #Person2# knows how to use the software, has computer qualifications, and can learn quite quickly.


In [17]:
i = 2

In [18]:
model_input = generate_input(summary, dialogue, i)
print(model_input)

Summary - #Person1# interviews #Person2# who has been a data entry operator for three years. #Person2# knows how to use the software, has computer qualifications, and can learn quite quickly.
Dialogue - 
#Person1#: Do you have any experience working with a computer?
#Person2#: Yes. I have been a data entry operator for three years.
<mask>
#Person2#: I have working knowledge of Windows and Dos. Actually, I'm quite familiar with both Java and C Programming Languages.
#Person1#: Do you have any other computer qualifications?
#Person2#: I have an ACRE certificate, GRADE 2.
#Person1#: Do you know how to use a PC to process the management information?
#Person2#: I'm sorry to say I'm not familiar with processing management information, but I'm sure I could learn quite quickly. It can't be too difficult, and I've got a quick mind. I can handle any problem you give me.

Speaker - #Person1#
Overlap - 1, Total - 28
Length - M


In [19]:
output = generate_new(model_input)
print(output)

 Do you have any experience in using the software?


In [20]:
new_dialogue = generate_new_dialogue(dialogue, i, output)
print(new_dialogue)

#Person1#: Do you have any experience working with a computer?
#Person2#: Yes. I have been a data entry operator for three years.
#Person1#: Do you have any experience in using the software?
#Person2#: I have working knowledge of Windows and Dos. Actually, I'm quite familiar with both Java and C Programming Languages.
#Person1#: Do you have any other computer qualifications?
#Person2#: I have an ACRE certificate, GRADE 2.
#Person1#: Do you know how to use a PC to process the management information?
#Person2#: I'm sorry to say I'm not familiar with processing management information, but I'm sure I could learn quite quickly. It can't be too difficult, and I've got a quick mind. I can handle any problem you give me.


## Replace ONE utterance

In [77]:
list_len = len(dialogue_list)
list_len

12460

In [78]:
new_list = []

for i in range(list_len):
    dialogue = dialogue_list[i]
    summary = summary_list[i]
    
    dialogue_sep = dialogue.split('\n')
    length = len(dialogue_sep)
    
    j = random.randint(0, length-1)
    
    model_input = generate_input(summary, dialogue, j)
    output = generate_new(model_input)
    new_dialogue = generate_new_dialogue(dialogue, j, output)
    
    dct = {}
    dct['fname'] = 'extra_' + str(i)
    dct['dialogue'] = new_dialogue
    dct['summary'] = summary
    
    
    new_list.append(dct)
    
    if i%50 == 0:
        print(i)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

In [79]:
x = 12345

In [80]:
print(new_list[x]['dialogue'])

#Person1#: Do you think Mr. Becket would be qualified for this job?
#Person2#: Mr. Becket? I'm not sure. He is a nice fellow, of course, and easy to get along with. But I doubt his professional expertise. I want someone who can get the job done.


In [81]:
print(dialogue_list[x])

#Person1#: Who do you think should get the job? How about Mr. Becket?
#Person2#: Mr. Becket? I'm not sure. He is a nice fellow, of course, and easy to get along with. But I doubt his professional expertise. I want someone who can get the job done.


In [82]:
print(new_list[x]['summary'])

#Person2# doesn't think Mr. Becket is qualified for the job


In [83]:
print(summary_list[x])

#Person2# doesn't think Mr. Becket is qualified for the job


In [84]:
data[0]

{'fname': 'train_0',
 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.",
 'summary': "Mr. Smi

In [85]:
new_list[x]

{'fname': 'extra_12345',
 'dialogue': "#Person1#: Do you think Mr. Becket would be qualified for this job?\n#Person2#: Mr. Becket? I'm not sure. He is a nice fellow, of course, and easy to get along with. But I doubt his professional expertise. I want someone who can get the job done.",
 'summary': "#Person2# doesn't think Mr. Becket is qualified for the job"}

In [86]:
with open('data/dialogsum/dialogsum.extra.jsonl', 'w') as outfile:
    for entry in new_list:
        json.dump(entry, outfile)
        outfile.write('\n')

In [87]:
file_path = 'data/dialogsum/dialogsum.extra.jsonl'

data = []

with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

id_list       = [sample['fname'] for sample in data]
dialogue_list = [sample['dialogue'] for sample in data]
summary_list  = [sample['summary'] for sample in data]

In [88]:
print(dialogue_list[x])

#Person1#: Do you think Mr. Becket would be qualified for this job?
#Person2#: Mr. Becket? I'm not sure. He is a nice fellow, of course, and easy to get along with. But I doubt his professional expertise. I want someone who can get the job done.


In [90]:
print(summary_list[x])

#Person2# doesn't think Mr. Becket is qualified for the job


## Generate Fixed Utterance Lists

In [69]:
def utterance_list_from_length(length, mask_perc):
    list_sz = int(length * mask_perc)
    if list_sz < 1:
        list_sz = 1
    
    rand_list = np.arange(length)
    np.random.shuffle(rand_list)
    rand_list = rand_list[:list_sz].tolist()
    rand_list.sort()
    
    return rand_list

In [71]:
utterance_list_from_length(15, 0.4)

[2, 3, 6, 10, 11, 14]

In [72]:
list_len = len(dialogue_list)
list_len

12460

In [73]:
utterance_list_arr = []

for i in range(list_len):
    dialogue = dialogue_list[i]
    dialogue_sep = dialogue.split('\n')
    length = len(dialogue_sep)
    
    utterance_list = utterance_list_from_length(length, 0.4)
    
    utterance_list_arr.append(utterance_list)

In [74]:
utterance_list_arr

[[0, 3, 5, 7],
 [3, 5],
 [0, 4, 5, 6],
 [1, 4, 9, 10],
 [3, 4],
 [1, 5],
 [0, 5],
 [1, 2, 4, 5, 8, 14],
 [0, 2],
 [0, 6, 7, 10],
 [3, 4, 6, 7, 15, 16, 17],
 [1, 3, 8],
 [3],
 [0, 3, 4, 9],
 [2, 4, 6, 7],
 [2],
 [2],
 [0, 3],
 [2, 5, 7, 8],
 [0, 4],
 [2, 5, 7],
 [0, 3, 7],
 [1, 2, 3, 4],
 [0, 6],
 [6, 7, 8, 9],
 [2, 8, 9, 10],
 [0, 5, 6, 8],
 [1, 5],
 [0, 1, 4, 5],
 [1, 2, 3],
 [2, 4, 5, 6],
 [1, 2, 5, 7, 10, 13],
 [1],
 [2, 3, 9, 10],
 [2, 4, 5, 7, 9],
 [1, 2, 5, 10, 11, 12, 14],
 [1, 4, 5, 10],
 [1, 6, 7, 9],
 [1, 4, 7, 10],
 [3, 4, 6, 11],
 [2, 5, 7, 10],
 [5, 6],
 [0, 3, 4, 5],
 [0, 4],
 [1, 4],
 [3, 5, 6, 7],
 [3],
 [2, 5, 6],
 [0, 1, 6],
 [1, 2, 6],
 [3, 6, 7, 8, 11, 16],
 [1],
 [1, 3, 5, 8],
 [0],
 [2, 3],
 [0, 3, 7, 8, 10, 12],
 [1],
 [4, 6],
 [0, 4, 5, 8, 10, 13, 14, 18],
 [1],
 [1, 5],
 [0, 1, 4, 8],
 [0, 5, 6, 8],
 [0, 5, 7, 9],
 [0, 4, 5, 10, 12],
 [0, 4, 5],
 [5, 6, 8, 10],
 [2, 4],
 [0, 2, 4, 5],
 [0, 1, 6, 11, 12, 13],
 [2, 3, 4, 6],
 [0, 2, 5],
 [1, 5, 6],
 [1, 3],
 [1, 

In [78]:
# with open("fixed_utterance_list.json", "w") as f:
#     json.dump(utterance_list_arr, f)

## Replace 30-40%

In [22]:
with open("fixed_utterance_list.json") as f:
    fixed_utterance_list = json.load(f)

In [23]:
fixed_utterance_list

[[0, 3, 5, 7],
 [3, 5],
 [0, 4, 5, 6],
 [1, 4, 9, 10],
 [3, 4],
 [1, 5],
 [0, 5],
 [1, 2, 4, 5, 8, 14],
 [0, 2],
 [0, 6, 7, 10],
 [3, 4, 6, 7, 15, 16, 17],
 [1, 3, 8],
 [3],
 [0, 3, 4, 9],
 [2, 4, 6, 7],
 [2],
 [2],
 [0, 3],
 [2, 5, 7, 8],
 [0, 4],
 [2, 5, 7],
 [0, 3, 7],
 [1, 2, 3, 4],
 [0, 6],
 [6, 7, 8, 9],
 [2, 8, 9, 10],
 [0, 5, 6, 8],
 [1, 5],
 [0, 1, 4, 5],
 [1, 2, 3],
 [2, 4, 5, 6],
 [1, 2, 5, 7, 10, 13],
 [1],
 [2, 3, 9, 10],
 [2, 4, 5, 7, 9],
 [1, 2, 5, 10, 11, 12, 14],
 [1, 4, 5, 10],
 [1, 6, 7, 9],
 [1, 4, 7, 10],
 [3, 4, 6, 11],
 [2, 5, 7, 10],
 [5, 6],
 [0, 3, 4, 5],
 [0, 4],
 [1, 4],
 [3, 5, 6, 7],
 [3],
 [2, 5, 6],
 [0, 1, 6],
 [1, 2, 6],
 [3, 6, 7, 8, 11, 16],
 [1],
 [1, 3, 5, 8],
 [0],
 [2, 3],
 [0, 3, 7, 8, 10, 12],
 [1],
 [4, 6],
 [0, 4, 5, 8, 10, 13, 14, 18],
 [1],
 [1, 5],
 [0, 1, 4, 8],
 [0, 5, 6, 8],
 [0, 5, 7, 9],
 [0, 4, 5, 10, 12],
 [0, 4, 5],
 [5, 6, 8, 10],
 [2, 4],
 [0, 2, 4, 5],
 [0, 1, 6, 11, 12, 13],
 [2, 3, 4, 6],
 [0, 2, 5],
 [1, 5, 6],
 [1, 3],
 [1, 

In [24]:
new_list = []

for i in range(1000):
    dialogue = dialogue_list[i]
    summary = summary_list[i]
    
    dialogue_sep = dialogue.split('\n')
    length = len(dialogue_sep)
    
    utterance_list = fixed_utterance_list[i]
    
    for j in utterance_list:
        model_input = generate_input(summary, dialogue, j)
        output = generate_new(model_input)
        output = output.replace('\n'," ")
        dialogue = generate_new_dialogue(dialogue, j, output)
    
    dct = {}
    dct['fname'] = 'extra_' + str(i)
    dct['dialogue'] = dialogue
    dct['summary'] = summary
    
    new_list.append(dct)
    
    if i%50 == 0:
        print(i)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


In [25]:
x = 69

In [26]:
print(new_list[x]['dialogue'])

#Person1#: Hello. I would like to ask you about the customer service.
#Person2#: Yes, I have a problem with the vacuum. It is broken.
#Person1#: Is it under warranty?
#Person2#: I think so. I bought it four months ago.
#Person1#: Yes, it is still covered by our warranty. Tell me the mode number of your vacuum, please.
#Person2#: Okay. The model number is 6594 - c.
#Person1#: What is the name of your office?
#Person2#: 906 Ottawa street. My name is David Yang. My phone number is 713-786-0234.
#Person1#: Okay. There are two Customer Service Offices in your area. The nearest one is Chadwick and Hacks Appliances.
#Person2#: Could you tell me where the office is located?
#Person1#: Sure. 878 Fennel South.
#Person2#: Okay. I will call them right away.
#Person1#: Please let me know when you have the time.
#Person2#: Okay. Thank you for your help.
#Person1#: My pleasure.


In [27]:
print(dialogue_list[x])

#Person1#: Hi. This is the Customer Service. How can I help you?
#Person2#: Hi. I bought one of your vacuums from spend-wart. It's broken now.
#Person1#: Is it under warranty?
#Person2#: I think so. I bought it four months ago.
#Person1#: Yes, it is still covered by our warranty. Tell me the mode number of your vacuum, please.
#Person2#: Okay. The model number is 6594 - c.
#Person1#: What's your address, your name and your phone number?
#Person2#: 906 Ottawa street. My name is David Yang. My phone number is 713-786-0234.
#Person1#: Okay. There are two Customer Service Offices in your area. The nearest one is Chadwick and Hacks Appliances.
#Person2#: Could you tell me where the office is located?
#Person1#: Sure. 878 Fennel South.
#Person2#: Oh, I know that place. It's only two minutes drive.
#Person1#: You have to call the office first.
#Person2#: All right. Thank you very much for your help.
#Person1#: My pleasure.


In [28]:
print(new_list[x]['summary'])

#Person2# phones the Customer Service because #Person2#'s vacuum's broken. #Person1# answers the phone, asks for more details, and tells #Person1# the location of the nearest Customer Service Office.


In [29]:
print(summary_list[x])

#Person2# phones the Customer Service because #Person2#'s vacuum's broken. #Person1# answers the phone, asks for more details, and tells #Person1# the location of the nearest Customer Service Office.


In [30]:
data[0]

{'fname': 'train_0',
 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.",
 'summary': "Mr. Smi

In [31]:
new_list[x]

{'fname': 'extra_69',
 'dialogue': '#Person1#: Hello. I would like to ask you about the customer service.\n#Person2#: Yes, I have a problem with the vacuum. It is broken.\n#Person1#: Is it under warranty?\n#Person2#: I think so. I bought it four months ago.\n#Person1#: Yes, it is still covered by our warranty. Tell me the mode number of your vacuum, please.\n#Person2#: Okay. The model number is 6594 - c.\n#Person1#: What is the name of your office?\n#Person2#: 906 Ottawa street. My name is David Yang. My phone number is 713-786-0234.\n#Person1#: Okay. There are two Customer Service Offices in your area. The nearest one is Chadwick and Hacks Appliances.\n#Person2#: Could you tell me where the office is located?\n#Person1#: Sure. 878 Fennel South.\n#Person2#: Okay. I will call them right away.\n#Person1#: Please let me know when you have the time.\n#Person2#: Okay. Thank you for your help.\n#Person1#: My pleasure.',
 'summary': "#Person2# phones the Customer Service because #Person2#'s v

In [32]:
with open('data/dialogsum/dialogsum.extra30-40.jsonl', 'w') as outfile:
    for entry in new_list:
        json.dump(entry, outfile)
        outfile.write('\n')

In [33]:
file_path = 'data/dialogsum/dialogsum.extra30-40.jsonl'

data = []

with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

id_list       = [sample['fname'] for sample in data]
dialogue_list = [sample['dialogue'] for sample in data]
summary_list  = [sample['summary'] for sample in data]

In [34]:
print(dialogue_list[x])

#Person1#: Hello. I would like to ask you about the customer service.
#Person2#: Yes, I have a problem with the vacuum. It is broken.
#Person1#: Is it under warranty?
#Person2#: I think so. I bought it four months ago.
#Person1#: Yes, it is still covered by our warranty. Tell me the mode number of your vacuum, please.
#Person2#: Okay. The model number is 6594 - c.
#Person1#: What is the name of your office?
#Person2#: 906 Ottawa street. My name is David Yang. My phone number is 713-786-0234.
#Person1#: Okay. There are two Customer Service Offices in your area. The nearest one is Chadwick and Hacks Appliances.
#Person2#: Could you tell me where the office is located?
#Person1#: Sure. 878 Fennel South.
#Person2#: Okay. I will call them right away.
#Person1#: Please let me know when you have the time.
#Person2#: Okay. Thank you for your help.
#Person1#: My pleasure.


In [35]:
print(summary_list[x])

#Person2# phones the Customer Service because #Person2#'s vacuum's broken. #Person1# answers the phone, asks for more details, and tells #Person1# the location of the nearest Customer Service Office.


## Replace 30-40% (FIXED SEQUENCE) First 500 (finetuned on 500-1k)